<a href="https://colab.research.google.com/github/RahmiaWati431/KematanganTomatMenggunakanMachineLearning/blob/main/UjiNVB60_40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
from IPython.display import display
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

# --- Mount Google Drive ---
drive.mount('/content/drive')

# --- Load Data NB ---
crop_df = pd.read_csv('/content/drive/MyDrive/Penelitian/Berkas/Crop/60:40/nvb/mcnemar/Prediksi_NB_Crop_60:40.csv')
nocrop_df = pd.read_csv('/content/drive/MyDrive/Penelitian/Berkas/TanpaCrop/60:40/nvb/mcnemar/Prediksi_NB_TanpaCrop_60:40.csv')

# --- Pastikan urutannya sama ---
crop_df.sort_values(by='Filename', inplace=True)
nocrop_df.sort_values(by='Filename', inplace=True)
crop_df.reset_index(drop=True, inplace=True)
nocrop_df.reset_index(drop=True, inplace=True)

# --- Pastikan label asli sama ---
assert all(crop_df['Actual'] == nocrop_df['Actual']), "Ground truth tidak cocok!"

# --- Buat tabel evaluasi ---
evaluasi_df = pd.DataFrame({
    'Label Asli': crop_df['Actual'],
    'NB Crop Prediksi': crop_df['Predicted_NB_Crop'],
    'NB TanpaCrop Prediksi': nocrop_df['Predicted_NB_Crop'],
})

# Tambahkan kolom kebenaran prediksi
evaluasi_df['Crop Benar?'] = evaluasi_df['Label Asli'] == evaluasi_df['NB Crop Prediksi']
evaluasi_df['TanpaCrop Benar?'] = evaluasi_df['Label Asli'] == evaluasi_df['NB TanpaCrop Prediksi']

# Fungsi untuk menentukan kategori McNemar
def tentukan_kategori(row):
    if row['Crop Benar?'] and not row['TanpaCrop Benar?']:
        return 'b (Crop benar, TanpaCrop salah)'
    elif not row['Crop Benar?'] and row['TanpaCrop Benar?']:
        return 'c (Crop salah, TanpaCrop benar)'
    else:
        return '-'

evaluasi_df['Kategori McNemar'] = evaluasi_df.apply(tentukan_kategori, axis=1)

# --- Tampilkan tabel ---
print("\n📋 Tabel Evaluasi Lengkap McNemar NB Crop vs TanpaCrop (80:20):")
display(evaluasi_df)

# --- Simpan tabel evaluasi ke Excel ---
output_excel = '/content/drive/MyDrive/Penelitian/Berkas/cropvsnocrop/Evaluasi_McNemar_NB_Crop_vs_TanpaCrop_60-40.xlsx'
evaluasi_df.to_excel(output_excel, index=False)
print(f"✅ Tabel evaluasi disimpan ke: {output_excel}")

# --- Hitung nilai b dan c untuk McNemar ---
b = sum((evaluasi_df['Crop Benar?']) & (~evaluasi_df['TanpaCrop Benar?']))
c = sum((~evaluasi_df['Crop Benar?']) & (evaluasi_df['TanpaCrop Benar?']))

# --- Buat tabel kontingensi ---
table = [[0, b],
         [c, 0]]

print("\n=== Tabel Kontingensi McNemar ===")
print(pd.DataFrame(table, index=['Crop Benar', 'Crop Salah'], columns=['TanpaCrop Salah', 'TanpaCrop Benar']))

# --- Uji McNemar ---
result = mcnemar(table, exact=True)
print(f"\nHasil Uji McNemar:")
print(f"Statistik: {result.statistic}")
print(f"p-value: {result.pvalue:.4f}")

# Interpretasi
alpha = 0.05
if result.pvalue < alpha:
    print("\nKesimpulan: Terdapat perbedaan signifikan antara model NB Crop dan NB TanpaCrop (p < 0.05)")
else:
    print("\nKesimpulan: Tidak terdapat perbedaan signifikan antara model NB Crop dan NB TanpaCrop (p >= 0.05)")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

📋 Tabel Evaluasi Lengkap McNemar NB Crop vs TanpaCrop (80:20):


,Label Asli,NB Crop Prediksi,NB TanpaCrop Prediksi,Crop Benar?,TanpaCrop Benar?,Kategori McNemar
0,Matang,Matang,SetengahMatang,True,False,"b (Crop benar, TanpaCrop salah)"
1,Matang,Matang,SetengahMatang,True,False,"b (Crop benar, TanpaCrop salah)"
2,Matang,Matang,SetengahMatang,True,False,"b (Crop benar, TanpaCrop salah)"
3,Matang,Matang,Matang,True,True,-
4,Matang,Matang,SetengahMatang,True,False,"b (Crop benar, TanpaCrop salah)"
...,...,...,...,...,...,...
79,Mentah,Mentah,Matang,True,False,"b (Crop benar, TanpaCrop salah)"
80,SetengahMatang,SetengahMatang,SetengahMatang,True,True,-
81,SetengahMatang,Matang,Matang,False,False,-
82,SetengahMatang,SetengahMatang,SetengahMatang,True,True,-


✅ Tabel evaluasi disimpan ke: /content/drive/MyDrive/Penelitian/Berkas/cropvsnocrop/Evaluasi_McNemar_NB_Crop_vs_TanpaCrop_60-40.xlsx

=== Tabel Kontingensi McNemar ===
            TanpaCrop Salah  TanpaCrop Benar
Crop Benar                0               18
Crop Salah                2                0

Hasil Uji McNemar:
Statistik: 2.0
p-value: 0.0004

Kesimpulan: Terdapat perbedaan signifikan antara model NB Crop dan NB TanpaCrop (p < 0.05)


# New Section